In [9]:
import torch
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from collections import deque
from tqdm import tqdm

from walker import PPO

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize environment
env = gym.make('Walker2d-v4', render_mode='rgb_array')

# Number of state and action
N_S = env.observation_space.shape[0]
N_A = env.action_space.shape[0]

# Initialize PPO model
ppo = PPO(N_S, N_A)

# Load the saved model
log_dir = "../runs/vessl/"
ppo.load(log_dir)

# Test the model and render in Jupyter Notebook
state, _ = env.reset()
done = False
total_reward = 0

while not done:
    action = ppo.actor_net.choose_action(torch.from_numpy(np.array(state).astype(np.float32)).unsqueeze(0))[0]
#     state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
#     action = ppo.actor_net.choose_action(state)
    state, reward, truncated, terminated, info = env.step(action)
    total_reward += reward
    done = truncated or terminated

    # Render the environment and display the frame in Jupyter Notebook
    frame = env.render()
    # plt.imshow(frame)
    # display(plt.gcf())
    # clear_output(wait=True)

print('Total reward:', total_reward)
env.close()

Total reward: -13.92208840538857
